# Problem 4
##  Largest Palindrome product
------

A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99.

*Find the largest palindrome made from the product of two 3-digit numbers.*

---
Correct result: **906609**

In [1]:
def is_palindrome(num):
    num_str = str(num)
    for n in range(0, len(num_str) // 2):
        if num_str[n] != num_str[-(n + 1)]:
            return False
    return True

def naive_method():
    largest_palindrome = -1
    factors = (-1, 1)
    for n in range(100, 1000):
        for m in range(500, 1000):
            if is_palindrome(n * m) and n * m > largest_palindrome:
                largest_palindrome = n * m
                factors = (n, m)
    return (largest_palindrome, factors)

def mult_11_method():
    largest_palindrome = -1
    factors = (-1, 1)
    for n in range(11, 1000, 11):
        for m in range(999, n - 1, -1):
            if is_palindrome(n * m) and n * m > largest_palindrome:
                largest_palindrome = n * m
                factors = (n, m)
    if largest_palindrome < 100000:
        largest_palindrome, factors = naive_method()
    return (largest_palindrome, factors)

def binned_method(nmax, assume_even_digits):
    multiple = 11 if assume_even_digits else 1
    largest_palindrome = -1
    factors = (-1, 1)
    for lb in range(nmax, 99, -1):
        prod_lb = lb ** 2
        if largest_palindrome != -1:
            break
        for n in range(nmax, lb - 1, -1):
            m = multiple * (lb // multiple)
            while m * n >= prod_lb and m > 0:
                condition = m * n > largest_palindrome and is_palindrome(m * n)
                if condition:
                    largest_palindrome = m * n
                    factors = (m, n)
                m -= multiple
    return (largest_palindrome, factors)

def combined_method(nmax=999):
    result = binned_method(nmax, True)
    return result if result[0] != -1 else binned_method(nmax, False)

### Discussion

The naive approach to this problem is to simply compare all pairs of numbers between the given upper and lower bounds. However, it can be proven that all palindromic numbers that have an even number of digits are divisible by 11. (See e.g. [jwilson.coe.uga.edu/emt669/Student.Folders/Bacchus.Mohamed/pal/pal.html](http://jwilson.coe.uga.edu/emt669/Student.Folders/Bacchus.Mohamed/pal/pal.html)). This result can be used to reduce the running time by an order of magnitude by letting one of our indices increment by 11 when checking for palindromes, provided that the largest palindrome for which we are checking does indeed have an even number of digits. By the results for the naive approach we can see that this is in fact true, but if we did not already know the answer, we could fall back to the simple approach in the case where no 6-digit palindrome were found.

Because we want to find the _largest_ palindrome product, we can further improve the algorithm by starting from the top and iterating downward. However, in order for this improvement to work we need to be able to order or group the pairs of numbers that we check so that we can know for certain that a particular palindrome is the largest without having to check any further pairs. A simple pair of _for_ loops with n running from upper bound to lower bound in the outer loop and m doing the same in the inner loop is not guaranteed to find a larger palindrome product before any smaller ones. In other words, a loop invariant is needed that assures that the largest palindrome product found in the current iteration of the loop (if any) will necessarily be larger than any found in subsequent loops.

A simple way to achieve this is to bin the pairs by their products, as looping over a decrementing lower bound for the products will achieve the loop invariant mentioned above. We can check all pairs such that $m \cdot n \ge {lb}^2$ before continuing to the next lower bound.

This approach is also an order of magnitude faster than the naive method. They can then be combined to take advantage of both improvements.

In [2]:
# Running and timing the alternative approaches:
from utils import computation_timer

results = computation_timer({'name': 'Naive Method', 'func': naive_method},
                            {'name': 'Multiples of 11 Method', 'func': mult_11_method},
                            {'name': 'Binned Method', 'func': lambda: binned_method(999, False)},
                            {'name': 'Combined Method', 'func': lambda: combined_method(999)})
print("Timed Results:")
for result in results:
    print("\t%s:" % result['name'])
    print("\t\tResult: %s, obtained in %f seconds" % (result['result'], result['running_time']))

Timed Results:
	Naive Method:
		Result: (906609, (913, 993)), obtained in 0.307025 seconds
	Multiples of 11 Method:
		Result: (906609, (913, 993)), obtained in 0.030369 seconds
	Binned Method:
		Result: (906609, (913, 993)), obtained in 0.015044 seconds
	Combined Method:
		Result: (906609, (913, 993)), obtained in 0.001638 seconds
